<a href="https://colab.research.google.com/github/SamirGen/ProjetPAVA-IA/blob/main/LegoRound3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt -y install libportaudio2
!pip install -q --use-deprecated=legacy-resolver tflite-model-maker
!pip install -q pycocotools
!pip install -q opencv-python-headless==4.1.2.30
!pip uninstall -y tensorflow && pip install -q tensorflow==2.8.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 65.4 kB of archives.
After this operation, 223 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libportaudio2 amd64 19.6.0-1build1 [65.4 kB]
Fetched 65.4 kB in 1s (68.5 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 122349

In [2]:
import os
import numpy as np
import tensorflow as tf

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below 2.13.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're usin

In [12]:
spec = model_spec.get('efficientdet_lite0')

In [13]:
# split data into training and testing set
import os, random, shutil

os.mkdir('/Pava-detection/train')
os.mkdir('/Pava-detection/test')

image_paths = os.listdir('/images')
annotations_paths = os.listdir('/annotations')
random.shuffle(image_paths)

for i, image_path in enumerate(image_paths):
  if image_path.endswith("checkpoints")==False:
    if i < int(len(image_paths) * 0.8):
      shutil.copy(f'/images/{image_path}', '/Pava-detection/train/')
      shutil.copy(f'/annotations/{image_path.replace("jpeg", "xml")}', '/Pava-detection/train/')
    else:
      shutil.copy(f'/images/{image_path}', '/Pava-detection/test/')
      shutil.copy(f'/annotations/{image_path.replace("jpeg", "xml")}', '/Pava-detection/test/')

In [14]:
train_data=object_detector.DataLoader.from_pascal_voc("/images","/annotations",label_map={1: "ConstructionLego", 2: "ConstructionLego2", 3: "ConstructionLego3"})
test_data=object_detector.DataLoader.from_pascal_voc("/images","/annotations",label_map={1: "ConstructionLego", 2: "ConstructionLego2", 3: "ConstructionLego3"})

In [15]:
model = object_detector.create(train_data, model_spec=spec, epochs=50, batch_size=16, train_whole_model=True, validation_data=test_data)

Epoch 1/50
16/16 [==============================] - 277s 14s/step - det_loss: 1.7341 - cls_loss: 1.1250 - box_loss: 0.0122 - reg_l2_loss: 0.0632 - loss: 1.7973 - learning_rate: 0.0140 - gradient_norm: 1.1543 - val_det_loss: 1.5762 - val_cls_loss: 1.0278 - val_box_loss: 0.0110 - val_reg_l2_loss: 0.0632 - val_loss: 1.6393
Epoch 2/50
16/16 [==============================] - 197s 13s/step - det_loss: 1.4552 - cls_loss: 0.9031 - box_loss: 0.0110 - reg_l2_loss: 0.0632 - loss: 1.5184 - learning_rate: 0.0200 - gradient_norm: 1.6466 - val_det_loss: 1.3609 - val_cls_loss: 0.8461 - val_box_loss: 0.0103 - val_reg_l2_loss: 0.0632 - val_loss: 1.4241
Epoch 3/50
16/16 [==============================] - 193s 13s/step - det_loss: 1.1660 - cls_loss: 0.6616 - box_loss: 0.0101 - reg_l2_loss: 0.0632 - loss: 1.2292 - learning_rate: 0.0199 - gradient_norm: 1.6232 - val_det_loss: 1.4160 - val_cls_loss: 0.9429 - val_box_loss: 0.0095 - val_reg_l2_loss: 0.0632 - val_loss: 1.4792
Epoch 4/50
16/16 [================

In [16]:
model.evaluate(test_data)

5/5 [==============================] - 108s 15s/step



{'AP': 0.32996675,
 'AP50': 0.39537552,
 'AP75': 0.37406442,
 'APs': -1.0,
 'APm': -1.0,
 'APl': 0.32996675,
 'ARmax1': 0.36286908,
 'ARmax10': 0.3979749,
 'ARmax100': 0.43467903,
 'ARs': -1.0,
 'ARm': -1.0,
 'ARl': 0.43467903,
 'AP_/ConstructionLego': 0.0054298947,
 'AP_/ConstructionLego2': 0.8842497,
 'AP_/ConstructionLego3': 0.10022061}

In [17]:
model.export(export_dir='.')